In [ ]:
import numpy as np
from itertools import combinations
import re
from functools import reduce 
from collections import defaultdict
import math

class Moon:
    def __init__(self, position, name):
        self.start_position = position.copy()
        self.position = position
        self.name = name
        self.velocity = [0]*3
    
    def move(self):
        self.position = self.sum_vec_element_wise(self.position, self.velocity)
       
    @staticmethod
    def sum_vec_element_wise(vec1, vec2):
        return [sum(x) for x in zip(vec1, vec2)]
    
    def update_velocity(self, moon):
        diff_velocity =  np.clip([v2 - v1 for v1, v2 in zip(self.position, moon.position)], a_min=-1, a_max=1)
        self.velocity = self.sum_vec_element_wise(self.velocity, diff_velocity)
    
    def energy(self):
        return sum(np.absolute(self.position)) * sum(np.absolute(self.velocity))
    
    @staticmethod
    def vec_to_text(vec):
        return "<x = {:>3}, y = {:>3}, z = {:>3}>".format(*vec)
    
    def __str__(self):
        return "{}: pos = {}, vel = {}".format(self.name, self.vec_to_text(self.position), self.vec_to_text(self.velocity))

    

class Sytem_lunaire:
    def __init__(self):
        self.moons = []
        self.moon_pattern = re.compile(r"<x=(?P<x>-?\d+).*y=(?P<y>-?\d+).*z=(?P<z>-?\d+)>$")
        
    def add_moon(self, moon_string):
        self.add_moons([moon_string])
    
    def add_moons(self, moons_string):
        for moon_string in moons_string:
            position = list(map(int, self.moon_pattern.match(moon_string).groupdict().values()))
            self.moons.append(Moon(position, "Moon n°{}".format(len(self.moons))))
    
    def update_velocity_moons(self):
        for moon1, moon2 in combinations(self.moons, 2):
            moon1.update_velocity(moon2)
            moon2.update_velocity(moon1)
    
    def move_moons(self):
        for moon in self.moons:
            moon.move()
    
    def iteration(self):
        self.update_velocity_moons()
        self.move_moons()
    
    def __str__(self):
        s = ""
        for moon in self.moons:
            s = "{}{}\n".format(s, moon)
        return s
    
    def energy(self):
        return sum(moon.energy() for moon in self.moons)
    
    def revolution(self, axis):
        return all(moon.position[axis] == moon.start_position[axis] and moon.velocity[axis] == 0 for moon in self.moons)
    
    @staticmethod
    def lcm(a, b):
        return abs(a*b) // math.gcd(a, b)
    
    def part_1(self, nb_iteration=1000, debug=False):
        if debug:
            print(0)
            print(self)
        
        for i in range(nb_iteration):
            self.iteration()
            if debug:
                print(i+1)
                print(self)
                
        print(nb_iteration)
        print(self)
        return self.energy()
    
    def part_2(self):
        add_axis = [True]*3
        i = 0
        moons_axis_revolution = {}
        while(True):
            i += 1
            self.iteration()
            for axis in range(3):
                if self.revolution(axis):
                    if axis not in moons_axis_revolution:
                        moons_axis_revolution[axis] = i
                    elif add_axis[axis]:
                        moons_axis_revolution[axis] = i - moons_axis_revolution[axis]
                        add_axis[axis] = False
            
            if all(not add_ax for add_ax in add_axis):
                break
        
        
        return reduce((lambda x, y: self.lcm(x, y)), moons_axis_revolution.values()) 


In [ ]:
moons_string = [
    "<x=-7, y=17, z=-11>",
    "<x=9, y=12, z=5>",
    "<x=-9, y=0, z=-4>",
    "<x=4, y=6, z=0>"
]
s_l1 = Sytem_lunaire()
s_l1.add_moons(moons_string)
print(s_l1.part_1())
print(s_l1.part_2())

In [ ]:
moons_test =[
    "<x=-1, y=0, z=2>",
    "<x=2, y=-10, z=-7>",
    "<x=4, y=-8, z=8>",
    "<x=3, y=5, z=-1>",
]
s_l2 = Sytem_lunaire()
s_l2.add_moons(moons_test)
print(s_l2.part_1(10))
print(s_l2.part_2())

In [ ]:
moons_test =[
    "<x=-8, y=-10, z=0>",
    "<x=5, y=5, z=10>",
    "<x=2, y=-7, z=3>",
    "<x=9, y=-8, z=-3>",
]
s_l3 = Sytem_lunaire()
s_l3.add_moons(moons_test)
print(s_l3.part_1(100))
print(s_l3.part_2())

In [ ]:
lcm(lcm(18, 28), 44)

In [ ]:
def lcm(a, b):
        return abs(a*b) // math.gcd(a, b)